--- Day 21: Scrambled Letters and Hash ---

The computer system you're breaking into uses a weird scrambling function to store its passwords. It shouldn't be much trouble to create your own scrambled password so you can add it to the system; you just have to implement the scrambler.

The scrambling function is a series of operations (the exact list is provided in your puzzle input). Starting with the password to be scrambled, apply each operation in succession to the string. The individual operations behave as follows:

    swap position X with position Y means that the letters at indexes X and Y (counting from 0) should be swapped.
    swap letter X with letter Y means that the letters X and Y should be swapped (regardless of where they appear in the string).
    rotate left/right X steps means that the whole string should be rotated; for example, one right rotation would turn abcd into dabc.
    rotate based on position of letter X means that the whole string should be rotated to the right based on the index of letter X (counting from 0) as determined before this instruction does any rotations. Once the index is determined, rotate the string to the right one time, plus a number of times equal to that index, plus one additional time if the index was at least 4.
    reverse positions X through Y means that the span of letters at indexes X through Y (including the letters at X and Y) should be reversed in order.
    move position X to position Y means that the letter which is at index X should be removed from the string, then inserted such that it ends up at index Y.

For example, suppose you start with abcde and perform the following operations:

    swap position 4 with position 0 swaps the first and last letters, producing the input for the next step, ebcda.
    swap letter d with letter b swaps the positions of d and b: edcba.
    reverse positions 0 through 4 causes the entire string to be reversed, producing abcde.
    rotate left 1 step shifts all letters left one position, causing the first letter to wrap to the end of the string: bcdea.
    move position 1 to position 4 removes the letter at position 1 (c), then inserts it at position 4 (the end of the string): bdeac.
    move position 3 to position 0 removes the letter at position 3 (a), then inserts it at position 0 (the front of the string): abdec.
    rotate based on position of letter b finds the index of letter b (1), then rotates the string right once plus a number of times equal to that index (2): ecabd.
    rotate based on position of letter d finds the index of letter d (4), then rotates the string right once, plus a number of times equal to that index, plus an additional time because the index was at least 4, for a total of 6 right rotations: decab.

After these steps, the resulting scrambled password is decab.

Now, you just need to generate a new scrambled password and you can access the system. Given the list of scrambling operations in your puzzle input, what is the result of scrambling abcdefgh?


--- Part Two ---

You scrambled the password correctly, but you discover that you can't actually modify the password file on the system. You'll need to un-scramble one of the existing passwords by reversing the scrambling process.

What is the un-scrambled version of the scrambled password fbgdceah?


In [1]:
filepath = "..\\data\\input_day_21.txt"
test1 = "..\\test\\test21_1.txt"
test2 = "..\\test\\test21_2.txt"

In [2]:
def read_input(filepath):
    with open(filepath, 'r') as f:
        lines = f.readlines()
    
    return lines

In [3]:
def swap_positions(index_1, index_2, string):
    string_list = list(string)
    a, b = string_list[index_1], string_list[index_2]
    string_list[index_1] = b
    string_list[index_2] = a
    return "".join(string_list)

In [4]:
def swap_letters(letter_1, letter_2, string):
    index_1 = string.find(letter_1)
    index_2 = string.find(letter_2)
    string_list = list(string)
    string_list[index_1] = letter_2
    string_list[index_2] = letter_1
    return "".join(string_list)

In [5]:
def rotate_direction(direction, steps, string):
    N = len(string)
    steps = steps%N
    if direction == "right":
        string = string[-steps:] + string[:-steps]
    elif direction == "left":
        string = string[steps:]+string[:steps]
    return string

In [6]:
def rotate_letter(letter, string):
    index = string.find(letter)
    if index >= 4:
        index += 1
    index += 1
    return rotate_direction("right", index, string)

In [7]:
def derotate_letter(letter, string):
    index = string.find(letter)
    N = len(string)
    # I can't think of an elegant solution so just try all possible rotations 
    for i in range(N):
        possible_string = rotate_direction("right", i, string)
        if rotate_letter(letter, possible_string) == string:
            return possible_string
    

In [8]:
def reverse_positions(start, end , string):
    return string[:start] + string[start:end+1][::-1] + string[end+1:]

In [9]:
def move_position(original, target, string):
    string_list = list(string)
    a = string_list[original]
    string_list.remove(a)
    string_list.insert(target, a)
    return "".join(string_list)

In [10]:
def day21a(filepath, string="abcdefgh"):
    
    lines = read_input(filepath)
    
    for i, line in enumerate(lines):
        #print(string)
        #print(i, line)
        # swap checks first
        if "swap" in line:
            if "position" in line:
                index_1 = int(line.strip().split()[2])
                index_2 = int(line.strip().split()[-1])
                string = swap_positions(index_1, index_2, string)
            elif "letter" in line:
                letter_1 = line.strip().split()[2]
                letter_2 = line.strip().split()[-1]
                string = swap_letters(letter_1, letter_2, string)
        
        # check for rotations
        elif "rotate" in line:
            if line.strip().split()[1] in ["left", "right"]:
                steps = int(line.strip().split()[2])
                direction = line.strip().split()[1]
                #print(direction, steps)
                string = rotate_direction(direction, steps, string)
            elif "letter" in line:
                letter = line.strip().split()[-1]
                string = rotate_letter(letter, string)
        
        # check for reverse
        elif "reverse" in line:
            start = int(line.strip().split()[2])
            end = int(line.strip().split()[-1])
            string = reverse_positions(start, end, string)
        
        # check for move
        elif "move" in line:
            original = int(line.strip().split()[2])
            target = int(line.strip().split()[-1])
            string = move_position(original, target, string)
    print(string)
    return string

In [11]:
def day21b(filepath, string = "fbgdceah"):
    
    lines = read_input(filepath)
    new_direction = {"left": "right", "right": "left"}
    for i, line in enumerate(lines[::-1]):
        #print(string)
        #print(i, line)
        # swap checks first
        if "swap" in line:
            if "position" in line:
                index_1 = int(line.strip().split()[2])
                index_2 = int(line.strip().split()[-1])
                string = swap_positions(index_1, index_2, string)
            elif "letter" in line:
                letter_1 = line.strip().split()[2]
                letter_2 = line.strip().split()[-1]
                string = swap_letters(letter_1, letter_2, string)
        
        # check for rotations
        elif "rotate" in line:
            if line.strip().split()[1] in ["left", "right"]:
                steps = int(line.strip().split()[2])
                direction = line.strip().split()[1]
                direction = new_direction[direction]                
                #print(direction, steps)
                string = rotate_direction(direction, steps, string)
            elif "letter" in line:
                letter = line.strip().split()[-1]
                string = derotate_letter(letter, string)
        
        # check for reverse
        elif "reverse" in line:
            start = int(line.strip().split()[2])
            end = int(line.strip().split()[-1])
            string = reverse_positions(start, end, string)
        
        # check for move
        elif "move" in line:
            original = int(line.strip().split()[2])
            target = int(line.strip().split()[-1])
            string = move_position(target, original, string)
    print(string)
    return string

In [12]:
def test():
    
    # rotate directions tests
    string = "abcd"
    assert rotate_direction("right", 1, string) == "dabc"
    assert rotate_direction("right", 2, string) == "cdab"
    assert rotate_direction("left", 1, string) == "bcda"
    print("Passed string rotation checks")
    
    string = "abcde"
    assert swap_positions(0, 4, string) == "ebcda"
    
    
    assert day21a(test1, "abcde") == "decab"
    print("Passed all checks")

In [13]:
test()

Passed string rotation checks
decab
Passed all checks


In [14]:
day21a(filepath)

baecdfgh


'baecdfgh'

In [15]:
day21b(filepath)

cegdahbf


'cegdahbf'